In [1]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer=LancasterStemmer
import numpy as np
from tensorflow.python.framework import ops
import tflearn
import random

2023-02-25 16:45:49.572115: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
!pip install tflearn

In [5]:
import pickle
data=pickle.load(open('training_data','rb'))

In [6]:
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']


In [7]:
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [8]:

ops.reset_default_graph()
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
model.load('model.tflearn')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


2023-02-25 16:45:58.133249: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-25 16:45:58.277382: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:357] MLIR V1 optimization pass is not enabled


INFO:tensorflow:Restoring parameters from /Users/aditya/Downloads/Farmer-friendly-contextual-chatbot-master/model.tflearn


In [9]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [stemmer().stem(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words, show_details=False):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [10]:
error_thresh=0.25
def classify(sentence):
    results=model.predict([bow(sentence,words)])[0]
    results=[[i,r] for i,r in enumerate(results) if r>error_thresh]
    results.sort(key=lambda x:x[1],reverse=True)
    return_list=[]
    for r in results:
        return_list.append((classes[r[0]],r[1]))
    return return_list

In [11]:
context={}
def response(sentence,user_id='123',show_details=False):
    results=classify(sentence)
    if results:
        while results:
            for i in intents['intents']:
                    if i['tag'] == results[0][0]:
                        if 'context_set' in i:
                            if show_details: print('context',i['context_set'])
                            context[user_id]=i['context_set']
                            
                        if not 'context_filter' in i or (user_id in context and 'context_filter' in i and i['context_filter']==context[user_id]):
                            if show_details: print('tag:', i['tag'])
                            return random.choice(i['responses'])
            results.pop(0)

In [12]:
response('tell me about wheat protection')


'SPRAY OF ZINK SULPHATE 5 GRAM +UREA 10 GRAM PER LITER WATER'

In [13]:
response('how can i protect my crops')

'Hi there, how can I help?'

In [14]:
response('how can i protect my wheat')

In [15]:
response('how can i save my barley')

'Good to see you again'

In [18]:
response('tell me about cotton protection')

'SPRAY OF ZINK SULPHATE 5 GRAM +UREA 10 GRAM PER LITER WATER'

In [19]:
response("Hello")

'Hi there, how can I help?'

In [20]:
response("tell me about nutrients")

'SPRAY OF MULTIPLAX (MICRONUTRIENT) 4 ML PER LITER WATER'

In [21]:
response("TELL ME ABOUT TAKING SOIL SAMPLE")

'Dig out a wedge of soil about 6 to 8 inches deep, and set this wedge aside.. Now dig out a half-inch piece of soil from the hole and pour it into your bucket.'

In [22]:
response("TELL ME SOMETHING ABOUT NUTRIENT MANAGEMENT ?")

'SPRAY OF ZINK SULPHATE 5 GRAM +UREA 10 GRAM PER LITER WATER'

In [28]:
response("tell me about cotton pesticides")

'SPRAY OF SOLUBLE NPK  18:18:18 GRADE 10 GM/LTR WATER'